In [71]:
from ycharts_parsers import YchartsDataVar
import polars as pl
from glob import glob
from tqdm import tqdm
from os import listdir

In [72]:
YCH_DATA_TYPE = YchartsDataVar.REVENUE
PARQUETS_PATH = f"./data/ycharts/{YCH_DATA_TYPE.value}_parqs"
YCH_DF_PATH = f"./data/ycharts/"

In [73]:
len(listdir(PARQUETS_PATH))

2139

In [74]:
df_pl_res = [pl.read_parquet(fp).drop("index") for fp in tqdm(glob(f"{PARQUETS_PATH}/*.parquet"))]

100%|██████████| 2139/2139 [00:13<00:00, 152.97it/s]


In [75]:
df_merged = df_pl_res[0]
for df in tqdm(df_pl_res[1:]):
    df_merged = df_merged.join(df, on="date", how="outer")
df_merged = df_merged.sort("date")
print(df_merged.shape)

100%|██████████| 2138/2138 [00:05<00:00, 406.26it/s]

(471, 2140)


In [76]:
df = df_merged.to_pandas().set_index("date").loc["2009":"2022"].resample("1Y").mean()
df.index = df.index.shift(4, freq='MS') # type: ignore

In [77]:
# df_merged.write_parquet(f"{YCH_DF_PATH}/{YCH_DATA_TYPE.value}_final.parquet")
df.to_csv(f"{YCH_DF_PATH}/{YCH_DATA_TYPE.value}_final.csv")